# Что нам нужно?

* Метод, при помощи которого мы будем генерировать какое-то не существующее до этого изображение ([forvard/reverse diffusion](https://arxiv.org/abs/2006.11239))
* Способ соединить вместе текст и изображение ([text-image representation model](https://arxiv.org/abs/2103.00020))
* Что-то для сжатия изображений (SD is a LDM)([autoencoder](https://arxiv.org/abs/2112.10752))
* Способ задать направление для генерации ([U-net + attention](https://arxiv.org/abs/2112.10752))

![](./data/breaf_scheme.png)

## Метод, при помощи которого мы будем генерировать какое-то не существующее до этого изображение
В предыдущем докладе данного юнита мы подробно останавливались на том, что такое диффузия, и какой она бывает.
В связи с этим далее будет картинка-плэйсхолдер, которая должна освежить память слушателя. (A если этого не произойдёт, к предыдущему докладу всегда можно [вернуться](./data/from_scratch.ipynb))

![](./data/ddpm_plot.png)

## Способ соединить вместе текст и изображение
CLIP - Contrastive Language-Image Pre-training
![](./data/CLIP_scheme.png)

(1) В первой части схемы показан принцип дообучения энкодера для изображений (например, ResNet50 или ViT) и энкодера для текстов (GPT-like трансформер) в Contrastive стратегии для батча размера N. Из-за того, что используемое расстояние не симметрично, "расталкивать" представления нужно в обе стороны, что выливается в соответствие поставленной задачи минимизации следующей функции потерь:

$$\Large \ell^{(I\rightarrow T)}_i = -log\frac{e^{\frac{\langle I_i, T_i\rangle}{τ}}}{\sum_{k=1}^{N} e^{\frac{\langle I_i, T_k\rangle}{τ}}};$$




$$\Large \ell^{(T\rightarrow I)}_i = -log\frac{e^{\frac{\langle T_i, I_i\rangle}{τ}}}{\sum_{k=1}^{N} e^{\frac{\langle T_i, I_k\rangle}{τ}}}$$

$$\Large \mathcal{L} = \frac{1}{N} \sum_{i=1}^{N} (\lambda \ell^{(I\rightarrow T)}_i + (1 - \lambda) \ell^{(T\rightarrow I)}_i)$$

(2) - (3) Во второй и третьей частях показано, как полученный результат можно использовать для zero-shot предсказаний на своём датасете. То, почему это названо созданием линейного классификатора можно представить, если принять  $T_1 ... T_n$ за наборы весов нейронов линейного слоя на N нейронов, а представление $I_1$ за вход для классификатора.

<img src="./data/perf_clip.png" alt="Drawing" style="width: 400px;"/>

### Особенности и ограничения CLIP

* Датасет для обучения 400mil пар картинка + текстовое описание
* По данным авторов, версия Zero-Shot CLIP, упомянутая в оригинальной статье, достигает near SOTA (SOTA как правило получено при помощи supervised техник) результатов на большинстве датасетов в supervised задачах. Для того, чтобы приблизиться к SOTA, нужно всего лишь х1000 времени и данных, что, однако __"infeasible to train with current hardware"__
* Модель сталкивается с трудностями генерализации, когда видит что-то, чего не было в обучающей выборке __"While zero-shot CLIP generalizes well to many natural image distributions as investigated in Section 3.3, we’ve observed that zero-shot CLIP still generalizes poorly to data that is truly out-of-distribution for it. ... CLIP learns a high quality semantic OCR representation that performs well on digitally rendered text, which is common in its pre-training dataset, as evidenced by performance on Rendered SST2. However, CLIP only achieves 88% accuracy on the handwritten digits of MNIST. An embarrassingly simple baseline of logistic regression on raw pixels outperforms zero-shot CLIP."__
* Модель не подходит для генерации текстового описания изображения и получила некоторый Social Bias из обучающей выборки. __"CLIP is trained on text paired with images on the internet.These image-text pairs are unfiltered and uncurated and result in CLIP models learning many social biases."__

### Где может быть использован в LDM

* Генерация представления для текста (text-encoder)
* Генерация представления для изображения (image-encoder)
* Ранжирование изображений (DALLE использует CLIP также для упорядочивания сгенерированных изображений перед тем, как отдать их пользователю)

## Что-то для сжатия изображений
![](./data/шакал.jpeg)


Пространство пикселей является пространством очень высокой размерности (каждый пиксель == измерение), в нём очень долго и затратно работать, при этом достаточно малое число пикселей несет действительно важную информацию

Метод (может варьироваться в разных имплементациях LDM, как и почти любой другой компонент): строим отображение в пространство меньшей размерности, в котором, тем не менее, изображение не лишается своих свойств - представление в новом пространстве всё ещё является изображением в привычном для нас понимании этого слова. Таким образом процедура представляет собой получение скетча, который сохраняет в себе максимум информации из входа

Характерным моментом при обучении является подбор размерности, нужно соблюсти баланс между желанием максимального сжатия/ускорения/удешевления расчетов и сохранением детализации в получаемом пространстве

Ниже приведен график, на котором отражена динамика обучения моделей с различным значением downsampling factor, который представляет собой следующее отношение:

$$x ∈ R^{H×W×3};   z ∈ R^{h×w×c}$$


$$f = H/h = W/w$$

![](./data/spatial_downsampling.png)

## Способ задать направление для генерации

### Внимание-внимание!
Слайды позаимствованы из [презентации](https://scholar.harvard.edu/binxuw/classes/machine-learning-scratch/materials/stable-diffusion-scratch)

![](./data/slide_1.png)
![](./data/slide_2.png)
![](./data/slide_3.png)
![](./data/slide_4.png)
![](./data/slide_5.png)
![](./data/slide_6.png)
![](./data/slide_7.png)
![](./data/slide_8.png)
![](./data/slide_9.png)
![](./data/slide_10.png)